# CARNIVAL output analysis

This is a tutorial to present how the functions/analysis developed in the network_tools repo can be used.
The _data_ folder contains the output of CARNIVAL (edges and nodes information) run for two Differential Expression Analysis comparing two groups (C1 and C2) of Hepatoblastoma pacients against Normal Tumor. 
The t-values from the DEA where use to calculate Dorothea TFs and progeny scores.
These were then the input of the CARNIVAL, together with the input network (omnipath_200303_all_geneSymbol_noComplex.csv)

In [1]:
import os
import pandas as pd
import rpy2.rinterface
%load_ext rpy2.ipython

edgesC1 = pd.read_csv('data/HB_C1vsNT_CARNIVAL_edges.csv', header = 0)
edgesC2 = pd.read_csv('data/HB_C2vsNT_CARNIVAL_edges.csv', header = 0)

nodesC1 = pd.read_csv('data/HB_C1vsNT_CARNIVAL_nodes.csv', header = 0)
nodesC2 = pd.read_csv('data/HB_C2vsNT_CARNIVAL_nodes.csv', header = 0)
omnipathNetwork = pd.read_csv('data/omnipath_200303_all_geneSymbol_noComplex.csv', header = 0)

/Users/rosherbal/anaconda3/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


In [ ]:
# Here we can run python code directly

In [ ]:
%%R

# adding this %%R creates the enviroment to run R code.
# we can pass information from python to R easily with
# check this documentation to know more:
# https://www.linkedin.com/pulse/interfacing-r-from-python-3-jupyter-notebook-jared-stufft/

# Create adjacency matrices and compare them pair-wise

We can create adjacency matrices based on the inicial network feeded to CARNIVAL (_createAdjacencyMatrix_).
This 'framework' is useful to make the matrices comparison simpler.

Once the adjacency matrices are produced, they can be compared using _compareAdjacencies_.
This can compare two matrices based only on the occurency of the interaction in both matrices (weighted=F),
or it can take into account the weights during the comparison (weighted=T).
The output of this function is a list of 3 matrices:
    - sharedMTX: shared interactions between both matrices
    - uMtx1: unique interactions for matrix 1
    - uMtx2: unique interactions for matrix 2

In [2]:
%%R -i edgesC1 -i edgesC2 -i omnipathNetwork

# load the functions
source('compare_topology_adjacency.r')

# create adjacency matrices with sign (always; - inhibition, + activation) and weights (optional)
adMTX = lapply(list(edgesC1,edgesC2), createAdjacencyMatrix, scafoldNET=omnipathNetwork, weighted=T)
print("Adjacency matrix dimensions")
print(lapply(adMTX, dim))

# compare adjacency matrices based only in the existance of an interaction (weighted=F)
matComparison = compareAdjacencies(adjMAT1=adMTX[[1]], adjMAT2=adMTX[[2]], weighted=F)
print("Type of interactions per matrix (no weight taken into account)")
print(lapply(matComparison, table))

# compare adjacency matrices based on the interaction and its weight (weighted=T)
matComparisonWeight = compareAdjacencies(adjMAT1=adMTX[[1]], adjMAT2=adMTX[[2]], weighted=T)
print("Type of interactions per matrix (using weights)")
print(lapply(matComparisonWeight, table))
 

[1] "Adjacency matrix dimensions"
[[1]]
[1] 3075 3136

[[2]]
[1] 3075 3136

[1] "Type of interactions per matrix (no weight taken into account)"
$sharedMTX

  -1    0    1 
  34 7928   48 

$uMtx1

  -1    0    1 
  14 7979   17 

$uMtx2

  -1    0    1 
  15 7969   26 

[1] "Type of interactions per matrix (using weights)"
$sharedMTX

-100    0  100 
  16 7962   32 

$uMtx1

-100  -71  -70  -54  -50  -46  -30  -29  -23    0   26   30   31   46   47   54 
   6    3    1    8    1    9    1    2    1 7945    1    1    1    3    1    3 
  69   70   71  100 
   1    1    1   20 

$uMtx2

-100  -68  -60  -56  -52  -50  -48  -44  -25  -16  -14    0   12   16   32   44 
   5    1    1    1    3   10    5    1    4    1    1 7935    1    1    3    1 
  48   50   51   52   56   68  100 
   5    4    1   10    1    3   12 

